In [30]:
from mip import Model, xsum, BINARY, MAXIMIZE
from queue import Queue

def read(filename):
    with open(filename, 'r') as file:
        lista = file.readlines()
        lista = [x.strip().split() for x in lista]  # Remova espaços em branco e quebre as linhas
        lista = [[int(y) for y in x] for x in lista]
        variaveis = lista[0][0]
        num_restricoes = lista[0][1]  # Renomeado para evitar acentuação em variáveis
        objetivo = lista[1]
        restricoes = lista[2:]
    return variaveis, num_restricoes, objetivo, restricoes

def solve_relaxation(model, restricoes, objetivo, variaveis):
    # Utiliza o modelo passado como parâmetro para não recriar um novo
    model.objective = xsum(objetivo[i] * model.vars[i] for i in range(variaveis))

    for data in restricoes:
        model += xsum(data[i] * model.vars[i] for i in range(variaveis)) <= data[-1]

    model.optimize()

    if model.num_solutions:
        return model.objective_value, [v.x for v in model.vars]
    else:
        return None, None

# das variáveis não inteiras, qual é a que tem casa decimal mais próxima de 0.5
def branch(model, solucao, epsilon=0.0001):

    nao_binarias = [(i, abs(x - round(x))) for i, x in enumerate(solucao) if abs(x - round(x)) > epsilon]
    if not nao_binarias:
        return None

  # mais perto de 0.5
    nao_binarias.sort(key=lambda x: -abs(x[1] - 0.5))
    branchV = nao_binarias[0][0]
    return branchV

def branch_and_bound(filename):
    variaveis, num_restricoes, objetivo, restricoes = read(filename)

    queue = Queue()
    queue.put(([], {i: None for i in range(variaveis)}))  # Initial state

    best_solution = None
    best_objective = float('-inf')
    
    while not queue.empty():
        
        current_constraints, current_integrality = queue.get()
        
        m = Model(sense=MAXIMIZE)
        m.vars = [m.add_var(var_type=BINARY) for i in range(variaveis)]
        # Adiciona as restrições ao modelo
        for (var, val) in current_constraints:
            if val is not None:
                m += m.vars[var] == val
        
        objective_value, solution = solve_relaxation(m, restricoes, objetivo, variaveis)
        
        

        if objective_value is None or objective_value <= best_objective:
            continue
        
        if all(isinstance(i, int) for i in solution):
            best_solution = solution
            best_objective = objective_value
            continue
        
        branch_var = branch(m, solution)
        if branch_var is not None:
            for val in [0, 1]:
                new_constraints = current_constraints + [(branch_var, val)]
                new_integrality = current_integrality.copy()
                new_integrality[branch_var] = val
                queue.put((new_constraints, new_integrality))
                
        
    return objective_value, solution



In [31]:
optimal_value, optimal_solution = branch_and_bound('teste2.txt')
print("Optimal value:", optimal_value)
print("Optimal solution:", optimal_solution)

Optimal value: 24.0
Optimal solution: [0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0]


In [32]:
optimal_value, optimal_solution = branch_and_bound('teste1.txt')
print("Optimal value:", optimal_value)
print("Optimal solution:", optimal_solution)

Optimal value: 20.0
Optimal solution: [0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0]


In [33]:
optimal_value, optimal_solution = branch_and_bound('teste3.txt')
print("Optimal value:", optimal_value)
print("Optimal solution:", optimal_solution)

Optimal value: 19.0
Optimal solution: [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]


In [34]:
optimal_value, optimal_solution = branch_and_bound('teste4.txt')
print("Optimal value:", optimal_value)
print("Optimal solution:", optimal_solution)

Optimal value: 10.0
Optimal solution: [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
